# Modeling

So far... we have just under 700 scripts devolved into bag-of-words, along with ratings (from 1 to 10). Let's start modeling! 

We'll read in the datafile, create a training and a testing set, reduce dimensionality (fitting on the training set only to reduce data leakage), and then try some models. First we'll do a simple model that just predicts the mean of the ratings; then we'll try a random forest, linear regression, and gradient boosting.

We'll start by loading the data from the csv file, dividing into X (features) and y (ratings), and creating a training set and a testing set.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

ratingsScriptsML = pd.read_csv('ratingsAndBagOfWords.csv')

In [2]:
ratingsScriptsML.head()

,Unnamed: 0,00,000,10,100,101,102,103,104,105,...,yourselves,youth,zero,zip,zips,zone,zoo,zoom,zooms,averageRating
0,0,0,0,0,0,0,0,0,0,0,...,0,14,0,0,0,0,0,0,0,6.2
1,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4.3
2,2,1,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,7.6
3,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.8
4,4,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,2,7.3


In [3]:
ratingsScriptsML.drop('Unnamed: 0',axis = 1, inplace = True)

In [4]:
ratingsScriptsML.head()

,00,000,10,100,101,102,103,104,105,106,...,yourselves,youth,zero,zip,zips,zone,zoo,zoom,zooms,averageRating
0,0,0,0,0,0,0,0,0,0,0,...,0,14,0,0,0,0,0,0,0,6.2
1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.3
2,1,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,7.6
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.8
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,2,7.3


Our dependent variable - ratings - is a continuous variable. For the random forest, we need integers. Since the ratings as given are rounded to the tenths place, we'll multiply them by 10. This way, our ratings will be integers; they'll range from 10 to almost 100.

In [5]:
X = ratingsScriptsML.drop('averageRating', axis= 1)
y = ratingsScriptsML['averageRating']

y = y*10
y = y.astype(int)

X_tr, X_te, y_tr, y_te = train_test_split(X,y,test_size = .2, random_state = 42)

We reduced the matrix in EDA to see how correlated the SVD features were with the average rating, but we didn't save the reduced matrix. We didn't save it so we could train it on just the training data, not the whole dataset. It's time to reduce our matrix again with SVD!

In [6]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 27, random_state = 42)
svd.fit(X_tr) # make sure only fit to the training set
X_tr_transformed = svd.transform(X_tr)
X_te_transformed = svd.transform(X_te)

In [7]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import accuracy_score,log_loss
from matplotlib import pyplot
from sklearn.ensemble import RandomForestClassifier

Create a validation set to leave the testing to last

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_tr_transformed,y_tr,test_size = .2, random_state = 42)

First let's try a very simple model

In [11]:
import numpy as np

y_val_mean = [round(np.mean(y_train),0)] * len(y_val)

In [12]:
y_val_mean[0:5]

[60.0, 60.0, 60.0, 60.0, 60.0]

In [13]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [14]:
rm = rmse(y_val, y_val_mean)

print('Simple Average: Root Mean Squared Error = %.3f' % (rm))


Simple Average: Root Mean Squared Error = 13.823


Hm, that's pretty good already, but we do have room for improvement.

Let's try a random forest. 

In [15]:
def rfc_and_print_scores(X_tra, y_tra, X_va, y_va, n_estimators, max_depth,criterion):
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth = max_depth,criterion=criterion, random_state = 1,n_jobs=-1)
    model_res = clf.fit(X_tra, y_tra)
    y_pred = model_res.predict(X_va)

    rm = rmse(y_va, y_pred)

    print('Random Forest: RMSE=%.3f' % (rm))

In [16]:
rfc_and_print_scores(X_train, y_train, X_val, y_val,300,None,'gini')

Random Forest: RMSE=17.240


Our score has gotten worse.

Let's see what happens when we change the number of estimators - we don't have a lot of scripts

In [18]:
rfc_and_print_scores(X_train, y_train, X_val, y_val,150,None,'gini')

Random Forest: RMSE=17.251


That's nearly the same. What if we drop the number of estimators more?

In [20]:
rfc_and_print_scores(X_train, y_train, X_val, y_val,100,None,'gini')  

Random Forest: RMSE=16.953


It gets slightly better, but not better than the simple model.

What about linear regression?

In [21]:
lr = LinearRegression()

lr.fit(X_train,y_train)

y_pred_lr = lr.predict(X_val)

rm = rmse(y_val, y_pred_lr)

print('Linear Regression: RMSE=%.3f' % (rm))

Linear Regression: RMSE=14.114


Better than the random forest, but not as good as the simple model

We can try gradient boosting

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

In [46]:
def gbc_and_print_scores(X_tra, y_tra, X_va, y_va, n_e, l_r, m_f, m_d):
    gb = GradientBoostingClassifier(n_estimators=n_e, learning_rate = l_r, max_features=m_f, max_depth = m_d, random_state = 0)

    model_grad = gb.fit(X_tra, y_tra)
    y_pred = model_grad.predict(X_va)

    rm = rmse(y_va, y_pred)

    print('Gradient Booster: RMSE=%.3f' % (rm))

In [47]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gbc_and_print_scores(X_train, y_train, X_val, y_val,300,learning_rate,2,2)

    print("Learning rate: ", learning_rate)
    print()

Gradient Booster: RMSE=17.605
Learning rate:  0.05

Gradient Booster: RMSE=16.641
Learning rate:  0.1

Gradient Booster: RMSE=18.710
Learning rate:  0.25

Gradient Booster: RMSE=20.090
Learning rate:  0.5

Gradient Booster: RMSE=20.341
Learning rate:  0.75

Gradient Booster: RMSE=27.173
Learning rate:  1



The random forest and linear regression performed better, but none of the models so far performed as well as a simple model of using the mean to predict the rating.

Let's make sure we have the best random forest.

In [37]:
num_trees_list = [70, 80, 90, 100, 110, 120]
for tree in num_trees_list:
    print('Number of trees: ', tree)
    rfc_and_print_scores(X_train, y_train, X_val, y_val,tree,None,'gini')
    print()
    #130, 140, 150, 160, 170

Number of trees:  70
Random Forest: RMSE=17.419

Number of trees:  80
Random Forest: RMSE=17.463

Number of trees:  90
Random Forest: RMSE=17.018

Number of trees:  100
Random Forest: RMSE=16.953

Number of trees:  110
Random Forest: RMSE=17.158

Number of trees:  120
Random Forest: RMSE=17.245



Looks like 100 trees are the best.

Let's tune another hyper-parameter, max_depth

In [40]:
max_depths = [2, 5, 10, 15, None]
for depth in max_depths:
    print('Max depth: ', depth)
    rfc_and_print_scores(X_train, y_train, X_val, y_val,100,depth,'gini')

Max depth:  2
Random Forest: RMSE=16.740
Max depth:  5
Random Forest: RMSE=16.740
Max depth:  10
Random Forest: RMSE=18.160
Max depth:  15
Random Forest: RMSE=17.016
Max depth:  None
Random Forest: RMSE=16.953


Looks like limiting the depth of the trees to 5 gives us slightly better results.

Let's check the criterion, given our other hyperparameters.

In [41]:
crits = ['gini', 'entropy', 'log_loss']
for crit in crits:
    print('Criterion: ', crit)
    rfc_and_print_scores(X_train, y_train, X_val, y_val,100,5,crit)

Criterion:  gini
Random Forest: RMSE=16.740
Criterion:  entropy
Random Forest: RMSE=16.921
Criterion:  log_loss
Random Forest: RMSE=16.921


The default 'gini' score is the best, but it's still not better than linear regression or the simple model.

So the best parameters for the random forest are:<br>
-100 trees<br>
-a max-depth of 5<br>
-'gini' criterion

Let's tune the gradient booster and see if it gets better. We've already found that a learning rate of 0.1 gives the "best" result of an RMSE of 16.641

In [48]:
num_ests = [50,100,150,200,250,300]
for num_est in num_ests:
    gbc_and_print_scores(X_train, y_train, X_val, y_val,num_est,0.1,2,2)

    print("Number of estimators: ", num_est)
    print()

Gradient Booster: RMSE=17.576
Number of estimators:  50

Gradient Booster: RMSE=17.654
Number of estimators:  100

Gradient Booster: RMSE=16.805
Number of estimators:  150

Gradient Booster: RMSE=16.943
Number of estimators:  200

Gradient Booster: RMSE=16.214
Number of estimators:  250

Gradient Booster: RMSE=16.641
Number of estimators:  300



So 250 estimators is slightly better. Let's tune max features next.

In [50]:
max_features = [0.5,1,2,5,10]
for max_f in max_features:
    gbc_and_print_scores(X_train, y_train, X_val, y_val,250,0.1,max_f,2)

    print("Max features: ", max_f)
    print()

Gradient Booster: RMSE=17.415
Max features:  0.5

Gradient Booster: RMSE=16.992
Max features:  1

Gradient Booster: RMSE=16.214
Max features:  2

Gradient Booster: RMSE=26.073
Max features:  5

Gradient Booster: RMSE=20.288
Max features:  10



A max of 2 features is the best. Finally, we'll test max depth.

In [51]:
max_depths = [1,2,5,10]
for max_d in max_depths:
    gbc_and_print_scores(X_train, y_train, X_val, y_val,250,0.1,2,max_d)

    print("Max depths: ", max_d)
    print()

Gradient Booster: RMSE=17.556
Max depths:  1

Gradient Booster: RMSE=16.214
Max depths:  2

Gradient Booster: RMSE=17.366
Max depths:  5

Gradient Booster: RMSE=17.745
Max depths:  10



So the best we can do with a gradient booster is an RMSE of 16.214

Let's try a polynomial regression. 

We need to transform the X data further, so we'll also need to split our training data into a training and validation set again.

In [34]:
from sklearn.preprocessing import PolynomialFeatures
poly_transform = PolynomialFeatures(degree=2, include_bias=False)
X_poly_train = poly_transform.fit_transform(X_tr_transformed)
X_poly_test = poly_transform.transform(X_te_transformed)


X_poly_train, X_poly_val, y_poly_train, y_poly_val = train_test_split(X_poly_train,y_tr,test_size = .2, random_state = 42)

In [36]:
lr_poly = LinearRegression()

lr_poly.fit(X_poly_train, y_poly_train)

y_poly_pred_lr = lr_poly.predict(X_poly_val)

rm = rmse(y_poly_val, y_poly_pred_lr)

print('Polynomial Regression: RMSE=%.3f' % (rm))


Polynomial Regression: RMSE=121058.951


This model does much worse than the others tried so far.

We used 27 features in the SVD, because that's about the square root of the number of movies we have. However, there's also a suggestion that there should be 10 times the observations as number of features, so let's try refitting the SVD for 7 features.

In [42]:
svd7 = TruncatedSVD(n_components = 7, random_state = 42)
svd7.fit(X_tr) # make sure only fit to the training set
X_tr7_transformed = svd7.transform(X_tr)
X_te7_transformed = svd7.transform(X_te)

In [43]:
X7_train, X7_val, y7_train, y7_val = train_test_split(X_tr7_transformed,y_tr,test_size = .2, random_state = 42)

Our simple model, using the mean of the ratings to predict the ratings, won't change due to the change in features. As a reminder, that has a RMSE of 13.823. 

Let's try our optimal random forest, which had RMSE=16.740.

In [44]:
rfc_and_print_scores(X7_train, y7_train, X7_val, y7_val,100,5,'gini')

Random Forest: RMSE=16.854


The RMSE for the optimal random forest with 7 features is slightly worse. Let's try linear regression, which had a RMSE of 14.114.

In [45]:
lr7 = LinearRegression()

lr7.fit(X7_train,y7_train)

y7_pred_lr = lr7.predict(X7_val)

rm = rmse(y7_val, y7_pred_lr)

print('Linear Regression: RMSE=%.3f' % (rm))

Linear Regression: RMSE=13.917


This is better.

Let's try the optimal gradient booster (which, recall, was not very good).

In [52]:
gbc_and_print_scores(X7_train, y7_train, X7_val, y7_val,250,0.1,2,2)

Gradient Booster: RMSE=16.878


Still not very good.